In [93]:
SVG_CATEGORIES = [
    #1-6 doors
    {"color": [224, 62, 155], "isthing": 1, "id": 1, "name": "single door"},
    {"color": [157, 34, 101], "isthing": 1, "id": 2, "name": "double door"},
    {"color": [232, 116, 91], "isthing": 1, "id": 3, "name": "sliding door"},
    {"color": [101, 54, 72], "isthing": 1, "id": 4, "name": "folding door"},
    {"color": [172, 107, 133], "isthing": 1, "id": 5, "name": "revolving door"},
    {"color": [142, 76, 101], "isthing": 1, "id": 6, "name": "rolling door"},
    #7-10 window
    {"color": [96, 78, 245], "isthing": 1, "id": 7, "name": "window"},
    {"color": [26, 2, 219], "isthing": 1, "id": 8, "name": "bay window"},
    {"color": [63, 140, 221], "isthing": 1, "id": 9, "name": "blind window"},
    {"color": [233, 59, 217], "isthing": 1, "id": 10, "name": "opening symbol"},
    #11-27: furniture
    {"color": [122, 181, 145], "isthing": 1, "id": 11, "name": "sofa"},
    {"color": [94, 150, 113], "isthing": 1, "id": 12, "name": "bed"},
    {"color": [66, 107, 81], "isthing": 1, "id": 13, "name": "chair"},
    {"color": [123, 181, 114], "isthing": 1, "id": 14, "name": "table"},
    {"color": [94, 150, 83], "isthing": 1, "id": 15, "name": "TV cabinet"},
    {"color": [66, 107, 59], "isthing": 1, "id": 16, "name": "Wardrobe"},
    {"color": [145, 182, 112], "isthing": 1, "id": 17, "name": "cabinet"},
    {"color": [152, 147, 200], "isthing": 1, "id": 18, "name": "gas stove"},
    {"color": [113, 151, 82], "isthing": 1, "id": 19, "name": "sink"},
    {"color": [112, 103, 178], "isthing": 1, "id": 20, "name": "refrigerator"},
    {"color": [81, 107, 58], "isthing": 1, "id": 21, "name": "airconditioner"},
    {"color": [172, 183, 113], "isthing": 1, "id": 22, "name": "bath"},
    {"color": [141, 152, 83], "isthing": 1, "id": 23, "name": "bath tub"},
    {"color": [80, 72, 147], "isthing": 1, "id": 24, "name": "washing machine"},
    {"color": [100, 108, 59], "isthing": 1, "id": 25, "name": "squat toilet"},
    {"color": [182, 170, 112], "isthing": 1, "id": 26, "name": "urinal"},
    {"color": [238, 124, 162], "isthing": 1, "id": 27, "name": "toilet"},
    #28:stairs
    {"color": [247, 206, 75], "isthing": 1, "id": 28, "name": "stairs"},
    #29-30: equipment
    {"color": [237, 112, 45], "isthing": 1, "id": 29, "name": "elevator"},
    {"color": [233, 59, 46], "isthing": 1, "id": 30, "name": "escalator"},

    #31-35: uncountable
    {"color": [172, 107, 151], "isthing": 0, "id": 31, "name": "row chairs"},
    {"color": [102, 67, 62], "isthing": 0, "id": 32, "name": "parking spot"},
    {"color": [167, 92, 32], "isthing": 0, "id": 33, "name": "wall"},
    {"color": [121, 104, 178], "isthing": 0, "id": 34, "name": "curtain wall"},
    {"color": [64, 52, 105], "isthing": 0, "id": 35, "name": "railing"},
    {"color": [0, 0, 0], "isthing": 0, "id": 36, "name": "bg"},
]
def set_color(semanticId):
    return SVG_CATEGORIES[semanticId]['color']

In [4]:
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
def svg_reader(svg_path):
    svg_list = list()
    try:
        tree = ET.parse(svg_path)
    except Exception as e:
        print("Read{} failed!".format(svg_path))
        return svg_list
    root = tree.getroot()
    for elem in root.iter():
        line = elem.attrib
        line["tag"] = elem.tag
        svg_list.append(line)
    return svg_list
def svg_writer(svg_list, svg_path):
    for idx, line in enumerate(svg_list):
        tag = line["tag"]
        line.pop("tag")
        if idx == 0:
            root = ET.Element(tag)
            root.attrib = line
        else:
            if "}g" in tag:
                group = ET.SubElement(root, tag)
                group.attrib = line
            else:
                node = ET.SubElement(group, tag)
                node.attrib = line
    prettyxml = BeautifulSoup(ET.tostring(root, "utf-8"), "xml").prettify()
    with open(svg_path, "w") as f:
        f.write(prettyxml)
def cvt_all_color(svg_path,semanticIds, output_dir=None):
    """Convert line color into black to align with other floorplan dataset"""
    tmp = svg_reader(svg_path)
    id=0
    for _, line in enumerate(tmp):
        if "stroke" in line.keys():
            rgb=set_color(semanticIds[id])
            line["stroke"] = f"rgb({rgb[0]},{rgb[1]},{rgb[2]})"
            line['semanticId']=semanticIds[id]
            id+=1
    if output_dir is None:
        svg_writer(tmp, svg_path)
    else:
        raise NotImplementedError
def cvt_line_color(svg_path,semanticIds, output_dir=None):
    """Convert line color into black to align with other floorplan dataset"""
    tmp = svg_reader(svg_path)
    id=0
    for _, line in enumerate(tmp):
        if "stroke" in line.keys() and 'semanticId' in line.keys() and 'instanceId' in line.keys():
            rgb=set_color(semanticIds[id])
            line["stroke"] = f"rgb({rgb[0]},{rgb[1]},{rgb[2]})"
            line['semanticId']=semanticIds[id]
            id+=1
    if output_dir is None:
        svg_writer(tmp, svg_path)
    else:
        raise NotImplementedError
def count_stroke(svg_path):
    tree = ET.parse(svg_path)
    root = tree.getroot()

    elements_with_stroke = root.findall('.//*[@stroke]')
    return len(elements_with_stroke)

In [30]:
def seg1_line(semantic_pred,semantic_gt):
    out=[]
    for idx,gt in enumerate(semantic_gt):
        if gt!=35:
            out.append(semantic_pred[idx])
    return out
def seg2_all(semantic_pred,num):
    return semantic_pred[:num]

path='/home/jesse/Project/SymPoint/dataset/test/test/svg_gt/0612-0299.svg'
path='/home/jesse/Project/SymPoint/dataset/test/test/svg_gt/1332-0021.svg'
out_dir=''
semantic_pred=[]
semantic_gt=[]
#Test1
semanticIds = seg1_line(semantic_pred,semantic_gt)
cvt_line_color(path,semanticIds,out_dir)
#Test2
stroke_num = count_stroke(path)
semanticIds=seg2_all(semantic_pred,stroke_num)
cvt_all_color(path,semanticIds,out_dir)


223

In [5]:
path='/home/jesse/Project/SymPoint/dataset/test/test/svg_gt/1332-0021.svg'
tmp = svg_reader(path)
svg_writer(tmp,'test.svg')

In [5]:
import svgpathtools
import numpy as np
from stl import mesh

def convert_svg_to_stl(svg_file, stl_file):
    # Load SVG file and parse the paths
    paths, attributes = svgpathtools.svg2paths(svg_file)

    # Create STL mesh object
    mesh_data = mesh.Mesh(np.zeros(0, dtype=mesh.Mesh.dtype))

    for path in paths:
        # Convert SVG path to a series of line segments
        line_segments = path.to_polygons()

        for polygon in line_segments:
            # Create triangles from the polygon
            for i in range(len(polygon) - 2):
                triangle = np.array([
                    polygon[0],
                    polygon[i + 1],
                    polygon[i + 2]
                ])

                # Add the triangle to the STL mesh
                mesh_data.append(mesh.Mesh(np.array([triangle])))

    # Save the STL mesh to file
    mesh_data.save(stl_file)



# Example usage
svg_file = "/home/jesse/Project/SymPoint/data/out/test/gt/0001-0023.svg"  # Input SVG file path
stl_file = "output.stl"  # Output STL file path

convert_svg_to_stl(svg_file, stl_file)

AttributeError: 'Path' object has no attribute 'to_polygons'

In [10]:
import ezdxf

def create_square(filename):
    doc = ezdxf.new()
    msp = doc.modelspace()

    # 正方形的四个顶点坐标
    vertices = [
        (100, 100),
        (300, 100),
        (300, 200),
        (100, 200)
    ]

    # 绘制正方形的边
    msp.add_lwpolyline(vertices)

    # 保存DXF文件
    doc.saveas(filename)

# 调用函数生成正方形的DXF文件
create_square('square.dxf')

In [36]:
import ezdxf

def create_square_and_cube(filename):
    doc = ezdxf.new()
    msp = doc.modelspace()

    # Square
    length = 100
    start_point = (100, 100,0)
    vertices = [
        start_point,
        (start_point[0] + length, start_point[1],0),
        (start_point[0] + length, start_point[1] + length,0),
        (start_point[0], start_point[1] + length,0)
    ]

    # Create lines for the square
    for i in range(4):
        start = vertices[i]
        end = vertices[(i + 1) % 4]
        msp.add_line(start, end,dxfattribs={'layer': 'CUBE', 'color': 1})

    # Cube
    height = 100
    
    vertices_top = [(v[0], v[1], 50) for v in vertices]
    
    # Create lines for the cube
    for i in range(4):
        start = vertices[i]
        end = vertices_top[i]
        msp.add_line(start, end,dxfattribs={'layer': 'CUBE', 'color': 1})

    # Connect the corresponding vertices
    for i in range(4):
        start = vertices_top[i]
        end = vertices_top[(i + 1) % 4]
        msp.add_line(start, end,dxfattribs={'layer': 'CUBE', 'color': 1})
    
    solid = msp.add_solid(
        dxfattribs={'layer': 'FILL1', 'color': 1},
        points=[vertices[0],vertices[1],vertices_top[1],vertices_top[0]]
    )
    solid = msp.add_solid(
        dxfattribs={'layer': 'FILL2', 'color': 2},
        points=[vertices[2],vertices[3],vertices_top[3],vertices_top[2]]
    )
    print(vertices)
    print(vertices_top)
    # Save DXF file
    doc.saveas(filename)

# Call the function to generate the square and cube DXF file
create_square_and_cube('square_and_cube.dxf')

[(100, 100, 0), (200, 100, 0), (200, 200, 0), (100, 200, 0)]
[(100, 100, 50), (200, 100, 50), (200, 200, 50), (100, 200, 50)]


In [98]:
SVG_CATEGORIES = [
    #1-6 doors
    {"color": [224, 62, 155], "isthing": 1, "id": 1, "name": "single door","height":12},
    {"color": [157, 34, 101], "isthing": 1, "id": 2, "name": "double door","height":12},
    {"color": [232, 116, 91], "isthing": 1, "id": 3, "name": "sliding door","height":12},
    {"color": [101, 54, 72], "isthing": 1, "id": 4, "name": "folding door","height":12},
    {"color": [172, 107, 133], "isthing": 1, "id": 5, "name": "revolving door","height":12},
    {"color": [142, 76, 101], "isthing": 1, "id": 6, "name": "rolling door","height":12},
    #7-10 window
    {"color": [96, 78, 245], "isthing": 1, "id": 7, "name": "window","height":7},
    {"color": [26, 2, 219], "isthing": 1, "id": 8, "name": "bay window","height":7},
    {"color": [63, 140, 221], "isthing": 1, "id": 9, "name": "blind window","height":7},
    {"color": [233, 59, 217], "isthing": 1, "id": 10, "name": "opening symbol","height":7},
    #11-27: furniture
    {"color": [122, 181, 145], "isthing": 1, "id": 11, "name": "sofa","height":10},
    {"color": [94, 150, 113], "isthing": 1, "id": 12, "name": "bed","height":10},
    {"color": [66, 107, 81], "isthing": 1, "id": 13, "name": "chair","height":10},
    {"color": [123, 181, 114], "isthing": 1, "id": 14, "name": "table","height":10},
    {"color": [94, 150, 83], "isthing": 1, "id": 15, "name": "TV cabinet","height":10},
    {"color": [66, 107, 59], "isthing": 1, "id": 16, "name": "Wardrobe","height":10},
    {"color": [145, 182, 112], "isthing": 1, "id": 17, "name": "cabinet","height":10},
    {"color": [152, 147, 200], "isthing": 1, "id": 18, "name": "gas stove","height":10},
    {"color": [113, 151, 82], "isthing": 1, "id": 19, "name": "sink","height":10},
    {"color": [112, 103, 178], "isthing": 1, "id": 20, "name": "refrigerator","height":10},
    {"color": [81, 107, 58], "isthing": 1, "id": 21, "name": "airconditioner","height":10},
    {"color": [172, 183, 113], "isthing": 1, "id": 22, "name": "bath","height":10},
    {"color": [141, 152, 83], "isthing": 1, "id": 23, "name": "bath tub","height":10},
    {"color": [80, 72, 147], "isthing": 1, "id": 24, "name": "washing machine","height":10},
    {"color": [100, 108, 59], "isthing": 1, "id": 25, "name": "squat toilet","height":10},
    {"color": [182, 170, 112], "isthing": 1, "id": 26, "name": "urinal","height":10},
    {"color": [238, 124, 162], "isthing": 1, "id": 27, "name": "toilet","height":10},
    #28:stairs
    {"color": [247, 206, 75], "isthing": 1, "id": 28, "name": "stairs","height":10},
    #29-30: equipment
    {"color": [237, 112, 45], "isthing": 1, "id": 29, "name": "elevator","height":10},
    {"color": [233, 59, 46], "isthing": 1, "id": 30, "name": "escalator","height":10},

    #31-35: uncountable
    {"color": [172, 107, 151], "isthing": 0, "id": 31, "name": "row chairs","height":10},
    {"color": [102, 67, 62], "isthing": 0, "id": 32, "name": "parking spot","height":10},
    {"color": [167, 92, 32], "isthing": 0, "id": 33, "name": "wall","height":15},
    {"color": [121, 104, 178], "isthing": 0, "id": 34, "name": "curtain wall","height":15},
    {"color": [64, 52, 105], "isthing": 0, "id": 35, "name": "railing","height":10},
    {"color": [0, 0, 0], "isthing": 0, "id": 36, "name": "bg","height":10},
]

In [94]:
required_id=[1,2,3,4,5,6,7,8,9,10,28,33,34]
from svgpathtools import svg2paths
import svgpathtools
# 使用wsvg函数加载SVG文件
filename = '/home/jesse/Project/SymPoint/data/out/test/test/0001-0023.svg'  # 替换为你的SVG文件路径
paths, attrs = svg2paths(filename)
# print(paths)
# print(attrs)
# print(type(paths[0]))
# 遍历所有路径元素
lines=[]
arcs=[]
for id,path in enumerate(paths):
    # 打印路径的d属性值
    if 'semanticId' in attrs[id] and int(attrs[id]['semanticId']) in required_id:
        # print(path)
        # print(attrs[id])
        for item in path:
            if type(item)==svgpathtools.path.Line:
                lines.append({'elem':item,"attr":attrs[id],'id':attrs[id]['semanticId']})
            elif type(item)==svgpathtools.path.Arc:
                arcs.append({"elem":item,"attr":attrs[id],'id':attrs[id]['semanticId']})
print(lines[0])
print(arcs[0])

{'elem': Line(start=(32.81842550000002+53.34899949999999j), end=(17.818425500000018+53.34899949999999j)), 'attr': {'d': 'M 32.81842550000002,53.34899949999999 L 17.818425500000018,53.34899949999999', 'fill': 'none', 'instanceId': '11', 'semanticId': '7', 'stroke': 'rgb(96,78,245)', 'stroke-width': '0.1', 'tag': 'path'}, 'id': '7'}
{'elem': Arc(start=(87.8214255+89.74899949999997j), radius=(7.8+7.8j), rotation=0.0, large_arc=False, sweep=False, end=(80.22142550000001+81.95099949999997j)), 'attr': {'d': 'M 87.8214255,89.74899949999997 A 7.8,7.8 0.0 0,0 80.22142550000001,81.95099949999997', 'fill': 'none', 'instanceId': '1', 'semanticId': '1', 'stroke': 'rgb(224,62,155)', 'stroke-width': '0.1', 'tag': 'path'}, 'id': '1'}


In [101]:
import ezdxf
def addLine(msp,start,end,layer,color):
    msp.add_line(start, end,dxfattribs={'layer': layer, 'color': color})

def addArc(msp,center,radius,start_angle,end_angle,layer,color):
    msp.add_arc(
        dxfattribs={'layer': layer,'color':color},
        center=center,  # 圆心
        radius=radius,  # 半径
        start_angle=start_angle,  # 起始角度（0-360度）
        end_angle=end_angle  # 结束角度（0-360度）
    )

doc = ezdxf.new()
msp = doc.modelspace()
for item in lines:
    addLine(msp,(item['elem'].start.real,item['elem'].start.imag,0),(item['elem'].end.real,item['elem'].end.imag,0),SVG_CATEGORIES[int(item['id'])-1]['name'],1)
# for item in arcs:
#     addArc(msp,(item['elem'].start.real,item['elem'].start.imag),item['elem'].radius.real,0,90,'arc',2)
for item in lines:
    addLine(msp,(item['elem'].start.real,item['elem'].start.imag,SVG_CATEGORIES[int(item['id'])-1]['height']),(item['elem'].end.real,item['elem'].end.imag,SVG_CATEGORIES[int(item['id'])-1]['height']),SVG_CATEGORIES[int(item['id'])-1]['name']+' top',1)
for item in lines:
    addLine(msp,(item['elem'].start.real,item['elem'].start.imag,0),(item['elem'].start.real,item['elem'].start.imag,SVG_CATEGORIES[int(item['id'])-1]['height']),SVG_CATEGORIES[int(item['id'])-1]['name']+' inter',2)
    addLine(msp,(item['elem'].end.real,item['elem'].end.imag,0),(item['elem'].end.real,item['elem'].end.imag,SVG_CATEGORIES[int(item['id'])-1]['height']),SVG_CATEGORIES[int(item['id'])-1]['name']+' inter',2)
doc.saveas('test.dxf')